# Variable Neighborhood Search

In [1]:
# Library
import random
import job
import ordonnancement
import sommet
import flowshop
import time
import copy
import math

# Perturb

In [2]:
def voisin_aleatoire(ordo):                
    nb_jobs=len(ordo.sequence)
    sequence=copy.copy(ordo.sequence)
    k=random.randint(0, len(sequence)-1)
    i=random.randint(0, len(sequence)-1)
    while i==k:
        i=random.randint(0, len(sequence)-1)
    y1=sequence[k]
    y2=sequence[i]
    sequence[k]=y2
    sequence[i]=y1
    ordo2 = ordonnancement.Ordonnancement(ordo.nombre_machines)
    ordo2.ordonnancer_liste_job(sequence)
    return ordo2


def perturb( S_best , k ):
    COUNT = 0
    S = S_best
    
    while COUNT < k : 
        S = voisin_aleatoire(S)
        COUNT += 1
    return S 
        



# Local Search 

## Tabu

In [3]:
Iteration=15

def meilleur_voisin(ordo, Last_visited):                
    nb_jobs=len(ordo.sequence)
    best_ordo=ordo
    for k in range(nb_jobs-1):
        for i in range(k+1,nb_jobs):
            sequence=copy.copy(ordo.sequence)
            y1=sequence[k]
            y2=sequence[i]
            sequence[k]=y2
            sequence[i]=y1
            ordo2 = ordonnancement.Ordonnancement(ordo.nombre_machines)
            ordo2.ordonnancer_liste_job(sequence)
            if best_ordo==ordo and (ordo2.sequence not in Last_visited):
                best_ordo=ordo2
            if ordo2.duree<best_ordo.duree and (ordo2.sequence not in Last_visited):
                best_ordo=ordo2
    return best_ordo


def local_search_tabu(X_greedy, LIST_JOBS, NB_MACHINE):
    #Création d'une solution de départ
    Sc= X_greedy
    
    #S_best constitue la meilleure solution trouvée
    S_best=Sc
    
    #Initialisation de la liste des dernieres solutions trouvées
    Last_visited=[Sc.sequence]
    i=0
    while i < Iteration :
        Sc=meilleur_voisin(Sc, Last_visited)
        if Sc.duree<S_best.duree:
            S_best=Sc
        
        Last_visited.append(Sc.sequence)
        if len(Last_visited)>4:
            Last_visited.pop(0)
        i+=1
    return S_best




## Recuit simulé

In [4]:
def local_search_recuit_simule(S):
    #Création d'une solution de départ
    Sc= S
    
    #S_best constitue la meilleure solution trouvée
    S_best=Sc
    
    T=1
    Kb=1.3806*10**(-23)
    alpha=0.99
    #Initialisation de la liste des dernieres solutions trouvées
    Last_visited=[Sc.sequence]
    i=0
    while i < Iteration :
        S=voisin_aleatoire(Sc)
        
        if S.duree<Sc.duree:
            Sc=S
        else:
            p=random.random()
            if p<=math.exp((Sc.duree-S.duree)/(Kb*T)):
                Sc=S
        
        if Sc.duree<S_best.duree:
            S_best=Sc
        T=alpha*T
        i+=1
    return S_best

# Basic VNS

What is the criterion ?

In [5]:
def Basic_VNS( LIST_JOBS, NB_MACHINE ):
    
    # Initialization
    X = ordonnancement.Ordonnancement(NB_MACHINE)
    X.ordonnancer_liste_job(prob.ordre_NEH()) 
    S_best = X
    print('Initial solution : {} | Durée : {}'.format([x.numero for x in S_best.sequence], S_best.duree))
    
    k = 1
    k_max = 10 # Nombre de jobs ? 
    COUNT = 0
    
    while COUNT < 1000: 
        S_perturb = perturb(S_best, k )
        S = local_search_tabu(S_perturb , LIST_JOBS, NB_MACHINE) 
        #S = local_search_recuit_simule(S_perturb)
        
        if S.duree < S_best.duree : 
            S_best = S 
            print('New solution found : {} | Durée : {}'.format([x.numero for x in S_best.sequence], S_best.duree))
            k = 1
        else : 
            k+=1
            if k>k_max : 
                    k=1
        COUNT +=1
    return S_best
        
    

# Tests


In [6]:
print("JEU 1 :")
prob = flowshop.Flowshop()
prob.definir_par_fichier('data/jeu1.txt')
L = prob.liste_jobs
m = prob.nombre_machines


start_time = time.time()
result=Basic_VNS( L, m)
elapsed_time = time.time() - start_time
print('----- RESULTAT-----')
print('Temps de résolution : {}s'.format(round(elapsed_time,3)))
print("durée :", result.duree)
print([ job.numero for job in result.sequence])


JEU 1 :
Initial solution : [3, 2, 0, 1] | Durée : 54
----- RESULTAT-----
Temps de résolution : 1.298s
durée : 54
[3, 2, 0, 1]


In [105]:
print("JEU 2 :")
prob = flowshop.Flowshop()
prob.definir_par_fichier('data/jeu2.txt')
L = prob.liste_jobs
m = prob.nombre_machines

# Parameters
time_limit = 1

start_time = time.time()
result=Basic_VNS( L, m)
elapsed_time = time.time() - start_time
print('----- RESULTAT-----')
print('Temps de résolution : {}s'.format(round(elapsed_time,3)))
print("durée :", result.duree)
print([ job.numero for job in result.sequence])

JEU 2 :
Initial solution : [2, 7, 5, 3, 1, 0, 4, 6] | Durée : 705
New solution found : [2, 5, 0, 3, 1, 7, 4, 6] | Durée : 704
----- RESULTAT-----
Temps de résolution : 11.37s
durée : 704
[2, 5, 0, 3, 1, 7, 4, 6]


In [106]:
print("tai01 :")
prob = flowshop.Flowshop()
prob.definir_par_fichier('/Users/corentin/Desktop/Cours/2A/programmation_dynamique/projet/data/jeu_de_données_1/tai01.txt')
L = prob.liste_jobs
m = prob.nombre_machines


start_time = time.time()
result=Basic_VNS( L, m)
elapsed_time = time.time() - start_time
print('----- RESULTAT-----')
print('Temps de résolution : {}s'.format(round(elapsed_time,3)))
print("durée :", result.duree)
print([ job.numero for job in result.sequence])

tai01 :
Initial solution : [2, 16, 8, 7, 14, 13, 10, 15, 12, 18, 5, 3, 4, 17, 0, 1, 9, 6, 19, 11] | Durée : 1286
New solution found : [2, 14, 16, 7, 8, 13, 6, 10, 12, 15, 5, 4, 17, 18, 3, 1, 0, 9, 19, 11] | Durée : 1279
New solution found : [8, 14, 16, 5, 2, 7, 6, 10, 12, 15, 13, 4, 17, 18, 3, 1, 0, 9, 19, 11] | Durée : 1278


KeyboardInterrupt: 

In [107]:
print("tai11 :")
prob = flowshop.Flowshop()
prob.definir_par_fichier('/Users/corentin/Desktop/Cours/2A/programmation_dynamique/projet/data/jeu_de_données_1/tai11.txt')
L = prob.liste_jobs
m = prob.nombre_machines


start_time = time.time()
result=Basic_VNS( L, m)
elapsed_time = time.time() - start_time
print('----- RESULTAT-----')
print('Temps de résolution : {}s'.format(round(elapsed_time,3)))
print("durée :", result.duree)
print([ job.numero for job in result.sequence])

tai11 :
Initial solution : [17, 4, 1, 16, 2, 5, 11, 8, 14, 9, 19, 12, 7, 13, 18, 10, 3, 6, 0, 15] | Durée : 1680
New solution found : [17, 4, 1, 11, 9, 5, 18, 16, 8, 2, 12, 14, 7, 13, 19, 6, 10, 3, 0, 15] | Durée : 1623
New solution found : [17, 4, 8, 11, 9, 5, 18, 16, 1, 2, 13, 7, 19, 12, 14, 10, 3, 6, 0, 15] | Durée : 1612
New solution found : [17, 4, 8, 11, 3, 12, 7, 19, 16, 14, 2, 1, 9, 5, 18, 10, 13, 6, 0, 15] | Durée : 1596
New solution found : [17, 4, 8, 2, 16, 1, 14, 11, 3, 7, 19, 12, 9, 5, 18, 10, 13, 6, 0, 15] | Durée : 1594
New solution found : [17, 4, 8, 11, 16, 3, 12, 1, 2, 7, 19, 14, 9, 5, 18, 10, 13, 6, 0, 15] | Durée : 1592
New solution found : [17, 4, 8, 11, 16, 12, 2, 3, 1, 7, 19, 14, 9, 5, 18, 10, 13, 6, 0, 15] | Durée : 1591
New solution found : [17, 4, 8, 11, 16, 1, 2, 3, 7, 19, 12, 14, 9, 5, 18, 10, 13, 6, 0, 15] | Durée : 1588
New solution found : [17, 4, 8, 11, 16, 14, 2, 1, 9, 13, 3, 5, 18, 7, 19, 10, 12, 6, 0, 15] | Durée : 1586
New solution found : [17, 4, 1,

KeyboardInterrupt: 

In [7]:
print("tai51 :")
prob = flowshop.Flowshop()
prob.definir_par_fichier('/Users/corentin/Desktop/Cours/2A/programmation_dynamique/projet/data/jeu_de_données_1/tai51.txt')
L = prob.liste_jobs
m = prob.nombre_machines


start_time = time.time()
result=Basic_VNS( L, m)
elapsed_time = time.time() - start_time
print('----- RESULTAT-----')
print('Temps de résolution : {}s'.format(round(elapsed_time,3)))
print("durée :", result.duree)
print([ job.numero for job in result.sequence])

tai51 :
Initial solution : [34, 42, 19, 49, 44, 30, 4, 9, 32, 25, 5, 7, 35, 41, 28, 40, 45, 46, 27, 48, 31, 0, 12, 15, 33, 11, 47, 1, 37, 6, 38, 21, 16, 39, 10, 22, 13, 36, 20, 23, 17, 18, 8, 14, 3, 29, 43, 26, 24, 2] | Durée : 4082
New solution found : [34, 42, 30, 36, 44, 19, 4, 9, 32, 25, 5, 35, 7, 41, 28, 40, 45, 46, 48, 27, 31, 0, 12, 15, 1, 11, 47, 33, 37, 6, 38, 21, 20, 22, 10, 39, 13, 26, 16, 23, 17, 18, 8, 14, 29, 3, 43, 24, 49, 2] | Durée : 3976
New solution found : [34, 42, 30, 4, 35, 19, 44, 9, 32, 25, 5, 36, 7, 41, 28, 40, 45, 46, 48, 27, 31, 0, 12, 15, 1, 11, 47, 33, 37, 6, 38, 21, 20, 22, 10, 39, 13, 26, 16, 23, 17, 18, 14, 8, 29, 3, 43, 24, 49, 2] | Durée : 3975
New solution found : [34, 42, 30, 4, 35, 19, 44, 9, 32, 25, 5, 36, 7, 41, 28, 40, 45, 46, 48, 27, 31, 0, 12, 15, 1, 11, 47, 33, 37, 6, 38, 21, 20, 22, 10, 39, 13, 26, 16, 23, 17, 18, 8, 14, 29, 3, 43, 24, 49, 2] | Durée : 3972
New solution found : [34, 42, 30, 4, 35, 19, 9, 41, 32, 25, 5, 36, 33, 0, 28, 40, 45, 

KeyboardInterrupt: 